## Load Tensorflow and word2vec dictionary then sample

In [ ]:
import re
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.models.rnn import rnn_cell
from tensorflow.models.rnn import seq2seq
from nltk.tokenize import WhitespaceTokenizer
import nltk.data
from gensim.models import Word2Vec
import time
import yelp_util

In [ ]:
class Argument():
    save_dir = 'model/'
    rnn_size = 100
    num_layers = 2
    model = 'lstm'
    batch_size = 10
    seq_length = 25
    num_epochs = 50
    grad_clip = 5.
    learning_rate = 0.002
    decay_rate = 0.97
    n = 40
args = Argument()
args.vocab_size = 5965

In [ ]:
model = yelp_util.ReviewModel(args, True)

In [ ]:
# boiler plate
sess = tf.Session()
init = tf.initialize_all_variables()
sess.run(init)

In [ ]:
# restore session
saver = tf.train.Saver(tf.all_variables())
ckpt = tf.train.get_checkpoint_state(args.save_dir)
saver.restore(sess, ckpt.model_checkpoint_path)

In [ ]:
word2vec_model = Word2Vec.load('model/word2vec/yelp100_gensim') # load trained gensim model or dictionary

In [ ]:
def weighted_pick(weights):
    t = np.cumsum(weights)
    s = np.sum(weights)
    return(int(np.searchsorted(t, np.random.rand(1)*s)))

In [ ]:
prime="this restaurant is".split()
prime_ls = map(lambda x: word2vec_model.vocab[x].index, prime)
state_p = model.cell.zero_state(1, tf.float32).eval(session=sess)

word_id = prime_ls[-1]
num = 10

for word_id in prime_ls:
    x_p = np.zeros((1,1))
    x_p[0, 0] = word_id
    feed = {model.input_data : x_p, model.initial_state: state_p}
    [state_p] = sess.run([model.final_state], feed)

for n in xrange(num):
    x_p = np.zeros((1, 1))
    x_p[0, 0] = word_id
    feed = {model.input_data: x_p, model.initial_state:state_p}
    [probs, state] = sess.run([model.probs, model.final_state], feed)
    p = probs[0]
    # sample = int(np.random.choice(len(p), p=p))
    sample = weighted_pick(p)
    prime += [word2vec_model.index2word[sample]]
    word_id = sample